In [8]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 
import json

In [5]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities.csv")

cities_df = pd.read_csv("../../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

In [6]:
llama = results_df[results_df['model'] == 'llama3point1-instruct']
mistral = results_df[results_df['model'] == 'mistral-instruct']

In [7]:
llama

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Kahramanmaras', 'Varna']","I recommend Nalchik, Russia because of its be..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Thessaloniki', 'Kaunas', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Gaziantep', 'Sibiu', 'Thessaloniki...","I recommend Kaunas, Lithuania because it is a..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ..."
...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ..."
196,llama3point1-instruct,prompt_44_gemini-ui,[],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ..."
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Malatya', 'Craiova', 'Thessaloniki', 'Munich...","I recommend Munich, Germany for its vibrant a...","['Gaziantep', 'Malatya', 'Craiova', 'Thessalon...",I recommend Munich and why I recommended it: ...
198,llama3point1-instruct,prompt_50_gemini-ui,"['London', 'Antalya', 'Innsbruck', 'Tallinn', ...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...


### What are the sustainability ranks of the recommended cities?

In [14]:
def get_ranks(df):
    city_s_ranks = []
    for i, row in df.iterrows():
        context_path = os.path.join(results_dir, row['model'], row['prompt_id'], 'cities.json')
        with open(context_path) as f:
            context = json.load(f)

        city_rank = []
        for city in ast.literal_eval(row['rec_cities_sustainable']):
            for i, rec_city in enumerate(context): 
                if city == rec_city['city']:
                    city_rank.append({
                        'city': city,
                        's-fairness-rank': i+1
                    })
        city_s_ranks.append(city_rank)

    df['sustainability_ranks'] = city_s_ranks
    return df


In [15]:
llama_ranks = get_ranks(llama)
llama_ranks

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_51883/442793268.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...,"[{'city': 'Vitoria-Gasteiz', 's-fairness-rank'..."
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...,"[{'city': 'Copenhagen', 's-fairness-rank': 5}]"
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Kahramanmaras', 'Varna']","I recommend Nalchik, Russia because of its be...","[{'city': 'Nalchik', 's-fairness-rank': 3}, {'..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Thessaloniki', 'Kaunas', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Gaziantep', 'Sibiu', 'Thessaloniki...","I recommend Kaunas, Lithuania because it is a...","[{'city': 'Kaunas', 's-fairness-rank': 6}, {'c..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ...","[{'city': 'Kaunas', 's-fairness-rank': 1}]"
...,...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ...","[{'city': 'Innsbruck', 's-fairness-rank': 3}]"
196,llama3point1-instruct,prompt_44_gemini-ui,[],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ...","[{'city': 'Stavanger', 's-fairness-rank': 7}]"
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Malatya', 'Craiova', 'Thessaloniki', 'Munich...","I recommend Munich, Germany for its vibrant a...","['Gaziantep', 'Malatya', 'Craiova', 'Thessalon...",I recommend Munich and why I recommended it: ...,"[{'city': 'Gaziantep', 's-fairness-rank': 3}, ..."
198,llama3point1-instruct,prompt_50_gemini-ui,"['London', 'Antalya', 'Innsbruck', 'Tallinn', ...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...,"[{'city': 'Varna', 's-fairness-rank': 3}]"


In [16]:
mistral_ranks = get_ranks(llama)
mistral_ranks

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_51883/442793268.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...,"[{'city': 'Vitoria-Gasteiz', 's-fairness-rank'..."
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...,"[{'city': 'Copenhagen', 's-fairness-rank': 5}]"
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Kahramanmaras', 'Varna']","I recommend Nalchik, Russia because of its be...","[{'city': 'Nalchik', 's-fairness-rank': 3}, {'..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Thessaloniki', 'Kaunas', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Gaziantep', 'Sibiu', 'Thessaloniki...","I recommend Kaunas, Lithuania because it is a...","[{'city': 'Kaunas', 's-fairness-rank': 6}, {'c..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ...","[{'city': 'Kaunas', 's-fairness-rank': 1}]"
...,...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ...","[{'city': 'Innsbruck', 's-fairness-rank': 3}]"
196,llama3point1-instruct,prompt_44_gemini-ui,[],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ...","[{'city': 'Stavanger', 's-fairness-rank': 7}]"
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Malatya', 'Craiova', 'Thessaloniki', 'Munich...","I recommend Munich, Germany for its vibrant a...","['Gaziantep', 'Malatya', 'Craiova', 'Thessalon...",I recommend Munich and why I recommended it: ...,"[{'city': 'Gaziantep', 's-fairness-rank': 3}, ..."
198,llama3point1-instruct,prompt_50_gemini-ui,"['London', 'Antalya', 'Innsbruck', 'Tallinn', ...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...,"[{'city': 'Varna', 's-fairness-rank': 3}]"


### For how many prompts do the responses differ when sustainability is added? 